In [1]:
from Bio import SeqIO
from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IP
from Bio.SeqUtils.ProtParam import ProteinAnalysis as PA
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
from collections import defaultdict
import pandas as pd
import ipywidgets as widgets
from ipywidgets import *

In [2]:
def listtostr(strlist):
    output = ""
    for word in strlist:
        output += word + "_"
    return(output[:len(output)-1])
def simplifyname(orgname):
    firstname = [orgname.split("_")[0][0]]
    lastname = orgname.split("_")[1:]
    fullname = firstname + lastname
    return(listtostr(fullname))

In [3]:
def getHistPercent(protstring):
    Hcount = 0
    for AA in protstring:
        if AA == "H":
            Hcount += 1
    return(Hcount/len(protstring))

def findHistChains(protstring):
    chainDict = {}
    chainlen = 1
    lastAA = ""
    for AA in protstring:
        if AA == "H":
            if lastAA == "H":
                chainlen += 1
        elif chainlen > 1 and lastAA == "H":
            #print(f"Chain found, current length {chainlen}")
            if str(chainlen) in chainDict:
                chainDict[str(chainlen)] = chainDict[str(chainlen)] + 1
            else:
                chainDict[str(chainlen)] = 1
            chainlen = 1
        lastAA = AA
    if chainlen > 1:
        if str(chainlen) in chainDict:
            chainDict[str(chainlen)] = chainDict[str(chainlen)] + 1
        else:
            chainDict[str(chainlen)] = 1
    return(chainDict)

def HistScore(protstring):
    last4 = ""
    last8 = ""
    CD4 = 0
    CD8 = 0
    score = 0
    for AA in protstring:
        #iterate last4 & last8
        if len(last4) < 4:
            last4 += AA
        else:
            last4 = last4[1:] + AA
        if len(last8) < 8:
            last8 += AA
        else:
            last8 = last8[1:] + AA
        
        #scan for non-chain H groupings 
        if getHistPercent(last4) >= 0.5 and CD4 < 1:
            score += 2
            CD4 = 4
        else:
            CD4 -= 1
        if getHistPercent(last8) >= 0.375 and CD8 < 1:
            score += 6
            CD8 = 8
        else:
            CD8 -= 1
    
    #scan for chains
    histChains = findHistChains(protstring)
    for key in histChains:
        score += histChains[key] * (int(key) ** 2)
    return(score)

In [4]:
#Create a selector for the intended size exclusion media
geldict = {
    "Q Media (Triethylamine +)" : -1,
    "S Media (Sulfite -)" : 1,
}
gelselect = Dropdown(options=geldict.keys(),description="Media")
display(gelselect)

pHslider = widgets.FloatSlider(value=7.0,min=0,max=14,step=0.1)
display(pHslider)

#testbutton = widgets.Button(description="Confirm")
#output = widgets.Output()
#display(testbutton, output)
#def on_button_clicked(b):
#    with output:
#        runpH = pHslider.value
#        gelcharge = int(geldict[gelselect.value])
    
#testbutton.on_click(on_button_clicked)



Dropdown(description='Media', options=('Q Media (Triethylamine +)', 'S Media (Sulfite -)'), value='Q Media (Tr…

FloatSlider(value=7.0, max=14.0)

In [66]:
#Generate a file path for the correct .fasta file
input_filename = input("Enter Input Name")
output_name = input("Enter Output Name")
datafile = os.path.join('data', input_filename)
#organism_name = input_filename.replace(".faa","")

Enter Input Name E_coli.faa
Enter Output Name E_coli_SpH5


In [67]:
data_folder = os.path.join('data', output_name)

if not os.path.isdir(data_folder):
    os.mkdir(data_folder)

In [68]:
unfiltered = []
wash = []
fractions = []
with open(datafile) as protfile:
    for record in SeqIO.parse(protfile,"fasta"):
        unfiltered.append(record)
        sequence = str(record.seq).replace("X","Q")
        protparams = PA(sequence)
        if protparams.charge_at_pH(pHslider.value)*int(geldict[gelselect.value]) < 0:
            wash.append(record)
        else:
            fractions.append(record)    

#data_name = gelselect.value[0] + str(pHslider.value).replace(".","_") + "_" + simplifyname(organism_name)
data_name = output_name
SeqIO.write(unfiltered,os.path.join(data_folder,data_name+"_total.faa"),"fasta")
SeqIO.write(wash,os.path.join(data_folder,data_name+"_wash.faa"),"fasta")
SeqIO.write(fractions,os.path.join(data_folder,data_name+"_fractions.faa"),"fasta")


658

In [69]:
#Converting modified fasta into a pd dataframe to show filter at work, preset file name used for simplicity

data = {
    'sequence' : [],
    'description' : [],
    f'charge at ph {pHslider.value}' : [],
    'ID' : []
}


protcharge = []
prot_desc = []
prot_name = []
prot_id = []
with open(os.path.join(data_folder,data_name+"_fractions.faa")) as protfile:
  for record in SeqIO.parse(protfile,"fasta"):
    sequence = str(record.seq)
    data['sequence'].append(sequence)
    protparams = PA(sequence)
    charge = round(protparams.charge_at_pH(pHslider.value),2)
    data[f'charge at ph {pHslider.value}'].append(charge)
    data['description'].append(record.description)
    data['ID'].append(record.id)


df = pd.DataFrame.from_dict(data)

df.sort_values(by=[f'charge at ph {pHslider.value}'], inplace=True, ascending = bool(int(geldict[gelselect.value]) == -1))
df.reset_index(drop=True, inplace=True)
df

,sequence,description,charge at ph 5.0,ID
0,MIAKHVPMRSLGKSDFAGLANYITDAQSKDHRLGHVQATNCEAGSI...,sp|Q00192|TRAI5_ECOLX Protein TraI OS=Escheric...,79.88,sp|Q00192|TRAI5_ECOLX
1,MSHADMSDSSGFNEAAAAFSWNGPKKAINPYLDPAEVAPFSALSNL...,sp|P21312|ENDOR_ECOLX Probable replication end...,67.44,sp|P21312|ENDOR_ECOLX
2,MIAKHVPMRSIKKSDFAELVKYITDEQGKTERLGHVRVTNCEANTL...,sp|Q00191|TRAI6_ECOLX Protein TraI OS=Escheric...,67.20,sp|Q00191|TRAI6_ECOLX
3,MPRRQILSSEEQERLLVIPDDEIILTRMCFLNEQDIALINKHRRPA...,sp|P08504|TNP5_ECOLX Transposase for transposo...,63.93,sp|P08504|TNP5_ECOLX
4,MPRRSILSAAERESLLALPDSKDDLIRHYTFNDTDLSIIRQRRGPA...,sp|P06694|TNP2_ECOLX Transposase for transposo...,61.59,sp|P06694|TNP2_ECOLX
...,...,...,...,...
653,MSKNKIMPWVDALPNVEATDFQARRDQIEATMAEAAELVKQAEELR...,sp|Q52278|KLEA1_ECOLX Protein KleA OS=Escheric...,0.31,sp|Q52278|KLEA1_ECOLX
654,MAYLDEIQLKEMGFKSVGENVKISDKASFYGCDNISIGNNVRIDDF...,"sp|Q9XCW3|VIOB_ECOLX dTDP-4-amino-4,6-dideoxy-...",0.30,sp|Q9XCW3|VIOB_ECOLX
655,MIKSVIAGAVAMAVVSFGVNNAAPTIPQGQGKVTFNGTVVDAPCSI...,sp|P04127|PAPA_ECOLX Pap fimbrial major pilin ...,0.26,sp|P04127|PAPA_ECOLX
656,MPIGNLGHNPNVNNSIPPAPPLPSQTDGAGGRGQLINSTGPLGSRA...,sp|P0DJ92|TIR3_ECOLX Translocated intimin rece...,0.24,sp|P0DJ92|TIR3_ECOLX


In [70]:
def param_of_interest(protdata):
    output = bool(HistScore(protdata) >= 4)
    return(output)

nfractions = 7
fraclen = round(len(df)/nfractions)
seqhits = {}
noise = 0.10
for n in range(nfractions - 1):
    fracrecords = []
    #Generate min/max range values
    fuzzymin = int(round(n*fraclen-noise*fraclen,0))
    fuzzymax = int(round((n+1)*fraclen + noise*fraclen,0))
    if fuzzymin < 0:
        fuzzymin = 0
    if fuzzymax > len(df):
        fuzzymax = len(df)
        
    tempdf = df.iloc[fuzzymin:fuzzymax]
    for index  in tempdf.index:
        record = SeqRecord(Seq(tempdf['sequence'][index]),id=str(tempdf["ID"][index]),description=tempdf["description"][index])
        fracrecords.append(record)
        if param_of_interest(tempdf['sequence'][index]) == True:
            #print("yes")
            if n+1 in seqhits:
                seqhits[n+1][0] += 1
                seqhits[n+1][1].append(index)
            else:
                seqhits[n+1] = [1,[index]]
    SeqIO.write(fracrecords,os.path.join(data_folder,data_name+"_fraction"+ str(n+1) + ".faa"),"fasta")

tempdf = df.iloc[(nfractions-1)*fraclen:]
remainderrecord = []
for index  in tempdf.index:
        record = SeqRecord(Seq(tempdf['sequence'][index]),id=str(tempdf["ID"][index]),description=tempdf["description"][index])
        remainderrecord.append(record)
        if param_of_interest(tempdf['sequence'][index]):
            if nfractions in seqhits:
                seqhits[nfractions][0] += 1
                seqhits[nfractions][1].append(index)
            else:
                seqhits[nfractions] = [1,[index]]
SeqIO.write(remainderrecord,os.path.join(data_folder,data_name+"_fraction"+ str(nfractions) + ".faa"),"fasta")
tempdf

,sequence,description,charge at ph 5.0,ID
564,MKEPKSNIRKLIDIGGLIGR,sp|P33789|TRAY8_ECOLX Relaxosome protein TraY ...,3.35,sp|P33789|TRAY8_ECOLX
565,MPQRTFLMMLIVICVTILCFVWMVRDSLCGLRLQQGNTVLVATLAY...,sp|P16477|PNDA2_ECOLX Protein PndA OS=Escheric...,3.34,sp|P16477|PNDA2_ECOLX
566,MKMTSIQQRAELHRQIWQIANDVRGSVDGWDFKQYVLGALFYRFIS...,sp|P10484|T1M1_ECOLX Type I restriction enzyme...,3.30,sp|P10484|T1M1_ECOLX
567,MKLSLMVAISKNGVIGNGPDIPWSAKGEQLLFKAITYNQWLLVGRK...,sp|P00382|DYR1_ECOLX Dihydrofolate reductase t...,3.28,sp|P00382|DYR1_ECOLX
568,MRIEIMIDKEQKISQSTLDALESELYRNLRPLYPKTVIRIRKGSSN...,sp|P21320|DINIR_ECOLX DinI-like protein in ret...,3.19,sp|P21320|DINIR_ECOLX
...,...,...,...,...
653,MSKNKIMPWVDALPNVEATDFQARRDQIEATMAEAAELVKQAEELR...,sp|Q52278|KLEA1_ECOLX Protein KleA OS=Escheric...,0.31,sp|Q52278|KLEA1_ECOLX
654,MAYLDEIQLKEMGFKSVGENVKISDKASFYGCDNISIGNNVRIDDF...,"sp|Q9XCW3|VIOB_ECOLX dTDP-4-amino-4,6-dideoxy-...",0.30,sp|Q9XCW3|VIOB_ECOLX
655,MIKSVIAGAVAMAVVSFGVNNAAPTIPQGQGKVTFNGTVVDAPCSI...,sp|P04127|PAPA_ECOLX Pap fimbrial major pilin ...,0.26,sp|P04127|PAPA_ECOLX
656,MPIGNLGHNPNVNNSIPPAPPLPSQTDGAGGRGQLINSTGPLGSRA...,sp|P0DJ92|TIR3_ECOLX Translocated intimin rece...,0.24,sp|P0DJ92|TIR3_ECOLX


In [ ]:
seqhits
for key in seqhits:
    print(f'There are {seqhits[key][0]} hits in fraction {key}')
    for hit in seqhits[key][1]:
        print(f'\tHit on index {hit}')

In [ ]:
seqhits